# Mental Health Text Classification
This notebook applies classic machine learning techniques to classify mental health-related text into categories such as Anxiety, Depression, and Suicidal.

We use:
- TF-IDF for text vectorization
- Multiple traditional ML models
- A unified training and evaluation function


In [1]:
import sys
sys.path.append('../src')

In [4]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.svm import LinearSVC
from sklearn.naive_bayes import MultinomialNB
from sklearn.dummy import DummyClassifier
from model import train_and_evaluate
